In [1]:
import pandas as pd

df=pd.read_csv('../chats_fevereiro.csv')
print(df['transcript'].iloc[9])

( 0s )speaker_00:  olá, sou  davi e estou à disposição para te atender no que você precisar!
( 4s )speaker_00:  para registro, o protocolo do seu atendimento é 12192645.
( 15s )speaker_00:  olá, raphael! tudo bem? é um prazer poder te atender.
( 19s )speaker_01:  olá! td bem e vc?acabei de pagar a fatura do meu cartão mas ele ainda ta bloqueado. conseguiria desoquear?
( 41s )speaker_01:  olá! td bem e vc?acabei de pagar a fatura do meu cartão mas ele ainda ta bloqueado. conseguiria desoquear?
( 2m 23s )speaker_00:  tudo sim, agradeço por perguntar
( 3m 8s )speaker_00:  em sistema nós não temos disponível a reativação ao informar o pagamento, mas vi que ele foi efetuado por débito em conta, por isso o desbloqueio ocorrerá de forma mais veloz
( 3m 55s )speaker_00:  ele pode demorar até 1 dia útil, mas, normalmente, é imediato se o pagamento for via débito em conta
( 5m 54s )speaker_01:  deu certo aqui
( 6m 4s )speaker_01:  não foi por débito em conta mas deu certo
( 6m 5s )speaker_01:  v

## Classic Triple extraction

In [5]:
!ls -lh ../

total 175M
-rw-rw-r-- 1 kiki kiki  12M Apr 29 16:43 atendimentos_rv_topic_summarization.csv
-rw-rw-r-- 1 kiki kiki 417K Jun  4 11:55 BTG_TAGS.ipynb
-rw-rw-r-- 1 kiki kiki  70M Apr 24 12:39 chats_fevereiro.csv
-rw-rw-r-- 1 kiki kiki  12M May  7 00:52 corpus.pkl
-rw-rw-r-- 1 kiki kiki  32M Apr 25 15:56 corpus_preprocessing.pkl
-rw-r--r-- 1 kiki kiki  32M May  6 18:20 docs_3m.txt
-rw-rw-r-- 1 kiki kiki 1,5M Apr 24 12:54 embeddings_visualization.ipynb
drwxrwxr-x 2 kiki kiki 4,0K Jun  3 11:02 graph
-rw-rw-r-- 1 kiki kiki    0 Apr 23 13:15 hier_clustering.ipynb
-rw-rw-r-- 1 kiki kiki 2,1K May  6 19:17 keywords.txt
-rw-rw-r-- 1 kiki kiki 5,9M Apr 30 14:28 resumos_chats_fevereiro2.p
-rw-rw-r-- 1 kiki kiki 2,1M Apr 29 20:35 resumos_chats_fevereiro.p
-rw-rw-r-- 1 kiki kiki 994K Apr 24 12:44 saida_text.csv
-rw-rw-r-- 1 kiki kiki 8,1M Apr 24 12:44 saida_text_v2.csv
drwxrwxr-x 5 kiki kiki 4,0K May  6 19:55 taxogen
-rw-rw-r-- 1 kiki kiki    0 Apr 24 12:35 test.txt
drwxrwxr-x 2 kiki kiki 4,0K Apr 30 

In [2]:
df.shape

(25605, 52)

### Etapa 1- Extracao de entidades

### Etapa 2 - Extracao de Relacoes

## Prompts

In [2]:
def get_prompts(max_knowledge_triplets,text):
    prompt_triple_extraction="""
    A text from bank costumer assistant chat in Portuguese is given. 
    Given the text, extract up to"""+ str(max_knowledge_triplets)+""" knowledge triplets 
    in the form of (subject, predicate, object). Avoid stopwords.
    ---------------------
    Examples:
    
    Text: boa tarde jeison minha fatura está constando como não paga, mas a minha é débito automática.
    Triplets:
    (fatura, consta, não paga)
    (fatura, definida, debito automático)
    Text:  renata, como estes dias são considerados feriados, o agendamento está para o próximo dia útil,
    ou seja, amanhã.
    nesta situação, encaminharei seu caso para o nosso time interno verificar o porquê consta “vencida”, a fim de
    realizar essa reparação o mais rápido possível e nisto, peço que mantenha o saldo em conta para realizarmos
    o pagamento
    Triplets:
    (pagamento em feriado, agendado, próximo dia util)
    (pagamento, necessário,saldo em conta )
    Text:ok então, mas no próprio aplicativo informa que não foi paga a fatura e que irá gerar encargos.
    Triplets:
    (aplicativo, informa, fatura não paga)
    (fatura não paga, ocasiona, encargos )
    Text:obrigada por aguardar, as faturas do dia 11/12 e 13 de fevereiro estão isentas de juros e multa se forem pagos até dia 14/11.
    Triplets:
    (fatura, paga até vencimento, isenta de juros e multa)
    (fatura,não pagas até vencimento, geram juros e multa)
    
    ---------------------
    Do not add entities with persons names. Do no add entities with dates and numbers not relevant to the meaning. 
    Only add entities that are relevant to understand the problems and possible solution to user's questions.
    When you encounter entities that look similar but are differently spelled (e.g. cartão and cartões),
    use one name for all occurences, when extracting them from text. Try to diversify the entities extracted. 
    You have the freedom to write entities that do not appear in the text, but are relevant to explain the problem and solution, like in the examples.
    
    ---------------------
    Get triples that explain: The User problem, the solution to the problem, the entities involved in the problem and solution.
    Retorne SOMENTE um json com as triplas extraídas na seguinte forma:
    {
        "triples":[
            {"subject":"fatura","predicate":"consta","object":"não paga"},
            {"subject":"fatura","predicate":"definida","object":"debito automático"}
        ]
    }
    """

    user_text=f"""
    Text: {text}

    Triplets:

    """
    return prompt_triple_extraction,user_text

In [3]:
def get_prompts(max_knowledge_triplets,text):
    prompt_triple_extraction="""
    A text from bank costumer assistant chat in Portuguese is given. 
    Given the text, extract up to """+str(max_knowledge_triplets)+""" knowledge triplets 
    in the form of (subject, predicate, object). Avoid stopwords.

    Do not add entities with persons names. Do no add entities with dates and numbers not relevant to the meaning. 
    Only add entities that are relevant to understand the problems and possible solution to user's questions.
    When you encounter entities that look similar but are differently spelled (e.g. cartão and cartões),
    use one name for all occurences, when extracting them from text. Try to diversify the entities extracted. 
    You have the freedom to write entities that do not appear in the text, but are relevant to explain the problem and solution, like in the examples.
    
    Get triples that explain: The User problem(first triple), the entities involved in the problem and solution, and  the solution to the problem(last triple).
    Assure that the triples have connections between them.
    Entities relevant: cliente, solução
    Relation examples: ter problema, estar, possuir, obter,etc

    ---------------------
    Examples:
    
    Chat:
    ( 0s )speaker_00:  olá, sou  davi e estou à disposição para te atender no que você precisar!
    ( 4s )speaker_00:  para registro, o protocolo do seu atendimento é 12192645.
    ( 15s )speaker_00:  olá, raphael! tudo bem? é um prazer poder te atender.
    ( 19s )speaker_01:  olá! td bem e vc?acabei de pagar a fatura do meu cartão mas ele ainda ta bloqueado. conseguiria desoquear?
    ( 2m 23s )speaker_00:  tudo sim, agradeço por perguntar
    ( 3m 8s )speaker_00:  em sistema nós não temos disponível a reativação ao informar o pagamento, mas vi que ele foi efetuado por
      débito em conta, por isso o desbloqueio ocorrerá de forma mais veloz
    ( 3m 55s )speaker_00:  ele pode demorar até 1 dia útil, mas, normalmente, é imediato se o pagamento for via débito em conta
    ( 5m 54s )speaker_01:  deu certo aqui
    ( 6m 4s )speaker_01:  não foi por débito em conta mas deu certo
    ( 6m 5s )speaker_01:  valeu

    
    Triplas:
    (cliente,tem problema, cartão)
    (cartão,esta, bloqueado)
    (atendente,não tem, reativação)
    (cartão,libera, pagamento)
    (pagamento,foi, débito em conta)
    (débito em conta,possui, desbloqueio mais rápido)
    (solução,obtida por, pagamento)
    
    Obrigatory: Make it a chain of triples that are connected to each other. 
    The Relation need to have at least one verb that connects the entities.

    Retorne SOMENTE um json com as triplas extraídas na seguinte forma:
    {
        "triples":[
            {"subject":"fatura","predicate":"consta","object":"não paga"},
            {"subject":"fatura","predicate":"definida","object":"debito automático"}
        ]
    }
    """

    user_text=f"""
    Text: {text}

    Triplets:

    """
    return prompt_triple_extraction,user_text

## LLama

In [10]:
import transformers
import torch

# model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"
model_id = "unsloth/llama-3-70b-Instruct-bnb-4bit"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/kiki/miniconda3/envs/llama3/lib/python3.11/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors.index.json:   0%|          | 0.00/331k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/7.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/6.99G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.75G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 23.66 GiB of which 102.06 MiB is free. Including non-PyTorch memory, this process has 22.27 GiB memory in use. Of the allocated memory 21.89 GiB is allocated by PyTorch, and 62.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [9]:
max_knowledge_triplets=5
text=df['transcript'].iloc[7]
prompt_triple_extraction,user_text=get_prompts(max_knowledge_triplets,text)

messages = [
    {"role": "system", "content": prompt_triple_extraction},
    {"role": "user", "content":text},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Here are the extracted knowledge triplets in the form of (subject, predicate, object):

{
"triples": [
    {"subject": "cadastro", "predicate": "gera", "object": "erro"},
    {"subject": "erro", "predicate": "para", "object": "ser finalizado"},
    {"subject": "thiago", "predicate": "tem", "object": "cadastro com erro"},
    {"subject": "captura de tela", "predicate": "envia", "object": "para o atendente"},
    {"subject": "documento", "predicate": "consta", "object": "pendência"},
    {"subject": "selfie", "predicate": "é", "object": "requerido"},
    {"subject": "thiago", "predicate": "envia", "object": "selfie e documento"},
    {"subject": "análise", "predicate": "é", "object": "realizada pelo time de especialistas"},
    {"subject": "informações", "predicate": "fornece", "object": "para o atendente"}
]
}

Note that I extracted entities that are relevant to the problem and solution, and avoided


In [4]:
import transformers
import torch

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

terminators


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[128001, 128009]

In [5]:
print(df['transcript'].iloc[7])

( 0s )speaker_00:  olá, sou  simone e estou à disposição para te atender no que você precisar!
( 3s )speaker_00:  para registro, o protocolo do seu atendimento é 12193240.
( 18s )speaker_00:  boa noite, thiago. tudo bem? é um prazer ter você aqui. verifiquei que informou no atendimento eletrônico abertura de conta, seria para essa questão que deseja o atendimento?
( 29s )speaker_01:  sim
( 1m 12s )speaker_00:  entendi!
( 1m 20s )speaker_00:  me conta, como posso te ajudar?
( 1m 39s )speaker_01:  meu cadastro só gera erro para ser finalizado 
( 1m 50s )speaker_00:  entendi!
( 2m 2s )speaker_00:  por favor, me informa seu nome completo e cpf?
( 2m 26s )speaker_01:  thiago de freitas lemos11817448730
( 3m 27s )speaker_00:  agradeço as informações!
( 3m 41s )speaker_00:  por favor, efetue uma captura de tela dessa mensagem de erro e me envie aqui no chat? assim consigo te orientar melhor. existe um botão para minimizar a aba do chat, desse modo, você consegue sair sem encerrá-lo e não perd

In [6]:
max_knowledge_triplets=5
text=df['transcript'].iloc[7]
prompt_triple_extraction,user_text=get_prompts(max_knowledge_triplets,text)
messages = [
    {"role": "system", "content": prompt_triple_extraction},
    {"role": "user", "content": user_text},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
result=outputs[0]["generated_text"][-1]
print(result['content'])

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


    Text: ( 0s )speaker_00:  olá, sou  simone e estou à disposição para te atender no que você precisar!
( 3s )speaker_00:  para registro, o protocolo do seu atendimento é 12193240.
( 18s )speaker_00:  boa noite, thiago. tudo bem? é um prazer ter você aqui. verifiquei que informou no atendimento eletrônico abertura de conta, seria para essa questão que deseja o atendimento?
( 29s )speaker_01:  sim
( 1m 12s )speaker_00:  entendi!
( 1m 20s )speaker_00:  me conta, como posso te ajudar?
( 1m 39s )speaker_01:  meu cadastro só gera erro para ser finalizado 
( 1m 50s )speaker_00:  entendi!
( 2m 2s )speaker_00:  por favor, me informa seu nome completo e cpf?
( 2m 26s )speaker_01:  thiago de freitas le


In [ ]:
1. (erro, gera, cadastro não finalizado)
2. (informação, precisa, nome completo e CPF)
3. (documento, precisa, selfie)
4. (análise, direcionado, time de especialistas)
5. (solicitação, direcionada, time de especialistas)

## GPT-4

In [5]:
# api_key=''

In [6]:
from openai import OpenAI

client = OpenAI(
  organization='',
    api_key=api_key
)

In [7]:
max_knowledge_triplets=5
text=df['transcript'].iloc[7]
prompt_triple_extraction,user_text=get_prompts(max_knowledge_triplets,text)
messages = [
    {"role": "system", "content": prompt_triple_extraction},
    {"role": "user", "content": user_text},
]
print(text)

( 0s )speaker_00:  olá, sou  simone e estou à disposição para te atender no que você precisar!
( 3s )speaker_00:  para registro, o protocolo do seu atendimento é 12193240.
( 18s )speaker_00:  boa noite, thiago. tudo bem? é um prazer ter você aqui. verifiquei que informou no atendimento eletrônico abertura de conta, seria para essa questão que deseja o atendimento?
( 29s )speaker_01:  sim
( 1m 12s )speaker_00:  entendi!
( 1m 20s )speaker_00:  me conta, como posso te ajudar?
( 1m 39s )speaker_01:  meu cadastro só gera erro para ser finalizado 
( 1m 50s )speaker_00:  entendi!
( 2m 2s )speaker_00:  por favor, me informa seu nome completo e cpf?
( 2m 26s )speaker_01:  thiago de freitas lemos11817448730
( 3m 27s )speaker_00:  agradeço as informações!
( 3m 41s )speaker_00:  por favor, efetue uma captura de tela dessa mensagem de erro e me envie aqui no chat? assim consigo te orientar melhor. existe um botão para minimizar a aba do chat, desse modo, você consegue sair sem encerrá-lo e não perd

In [8]:
stream = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

(cliente, tem problema, cadastro)

(cadastro, gera, erro)

(erro, é informado por, cliente)

(cliente, envia, documento e selfie)

(solução, é iniciada por, envio de documento e selfie)

## Gemini

In [10]:
api_key=''

In [11]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown

genai.configure(api_key=api_key)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [13]:
model = genai.GenerativeModel('gemini-1.5-pro-latest')

In [15]:
text=df['transcript'].iloc[7]
print(text)

( 0s )speaker_00:  olá, sou  simone e estou à disposição para te atender no que você precisar!
( 3s )speaker_00:  para registro, o protocolo do seu atendimento é 12193240.
( 18s )speaker_00:  boa noite, thiago. tudo bem? é um prazer ter você aqui. verifiquei que informou no atendimento eletrônico abertura de conta, seria para essa questão que deseja o atendimento?
( 29s )speaker_01:  sim
( 1m 12s )speaker_00:  entendi!
( 1m 20s )speaker_00:  me conta, como posso te ajudar?
( 1m 39s )speaker_01:  meu cadastro só gera erro para ser finalizado 
( 1m 50s )speaker_00:  entendi!
( 2m 2s )speaker_00:  por favor, me informa seu nome completo e cpf?
( 2m 26s )speaker_01:  thiago de freitas lemos11817448730
( 3m 27s )speaker_00:  agradeço as informações!
( 3m 41s )speaker_00:  por favor, efetue uma captura de tela dessa mensagem de erro e me envie aqui no chat? assim consigo te orientar melhor. existe um botão para minimizar a aba do chat, desse modo, você consegue sair sem encerrá-lo e não perd

In [16]:
max_knowledge_triplets=6

prompt_triple_extraction,user_text=get_prompts(max_knowledge_triplets,text)
text=f"{prompt_triple_extraction} \n {user_text}"
response = model.generate_content(text)
to_markdown(response.text)

> Here are some knowledge triplets extracted from the text:
> 
> 1. **(cliente, tem problema, cadastro)** 
>    * O cliente relata que está com problemas para finalizar seu cadastro.
> 2. **(cadastro, apresenta, erro)**
>    * O problema do cadastro se manifesta por meio de uma mensagem de erro. 
> 3. **(erro, impede, finalização)**
>    * A mensagem de erro impede que o cliente finalize o cadastro.
> 4. **(atendente, solicita, documentos)**
>    * Para solucionar o problema, a atendente solicita documentos do cliente:  captura de tela do erro, documento de identificação e selfie com o documento.
> 5. **(documentos, serão analisados por, time interno)** 
>    * Os documentos enviados pelo cliente serão analisados pelo time interno do banco.
> 6. **(solução, será enviada por,  e-mail)**
>    * A solução para o problema será enviada ao cliente por e-mail após análise. 


## Visualização

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
from helpers.df_helpers import documents2Dataframe


In [2]:
## Input data directory
data_dir = "cureus"
inputdirectory = Path(f"./data_input/{data_dir}")
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)

100%|██████████| 1/1 [00:01<00:00,  1.95s/it]

Number of chunks =  23
An extensive literature search was performed, and 56 articles published in peer-reviewed journals between 2005 and 2021 were selected and analyzed. The corresponding authors' experiential knowledge served as the foundation for the analysis.


In [3]:
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(23, 3)


,text,source,chunk_id
0,Abstract India’s health indicators have improv...,data_input/cureus/cureus-0015-00000040274.txt,185959646c7a47c5bc932429131ed527
1,"Categories: Public Health, Epidemiology/Public...",data_input/cureus/cureus-0015-00000040274.txt,67f4e9e125dd4bdc9ddef780a8371117
2,Introduction And Background India’s health ind...,data_input/cureus/cureus-0015-00000040274.txt,8c4d380f9eb542b1a20a14cf76a1dab8
3,"An extensive literature search was performed, ...",data_input/cureus/cureus-0015-00000040274.txt,dadf1c8e366d447d880a9eae422f67a2
4,Review Overview of the public and private heal...,data_input/cureus/cureus-0015-00000040274.txt,8498ba38ab4e40b19873d8000b34536a


In [4]:
import pickle

resumos = pickle.load(open("../resumos_chats_fevereiro.p", "rb"))

In [5]:
import pandas as pd
import random
df=pd.read_csv('../chats_fevereiro.csv')
text=df['transcript'].values
text=resumos
chunk_id=[str(random.randint(0,1000000))]*len(text)
source=['chat']*len(text)
df={'chunk_id':chunk_id,'text':text,'source':source}
df=pd.DataFrame(df)
df=df.iloc[:10]
df.head()

,chunk_id,text,source
0,44386,- O cliente entrou em contato com o atendimen...,chat
1,44386,- A cliente entrou em contato com o atendimen...,chat
2,44386,- O cliente entrou em contato com o atendimen...,chat
3,44386,- A atendente se identificou como Iracema e i...,chat
4,44386,"- O cliente tentou fazer um Pix de R$30,00, m...",chat


In [7]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

In [8]:
## To regenerate the graph with LLM, set this to True
regenerate = True
outputdirectory=Path(f"./data_output")
if regenerate:
    concepts_list = df2Graph(df, model='llama3')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

(68, 5)


,node_1,node_2,edge,chunk_id,count
0,análise,conta,está,44386,4
1,atendimento ao cliente,cliente,entrou em contato com,44386,4
2,atendente,mensagem de análise,solicitou um print da,44386,4
3,solicitação interna,análise,abrir uma,44386,4
4,e-mail,cliente,informar o sobre a liberação da conta por,44386,4


In [9]:
# import json
# import random

# import pickle
# triples=pickle.load(open('../triples.pkl','rb'))
# print(len(triples))

# dfg1={'node_1':[],'node_2':[],'edge':[],'count':[],'chunk_id':[]}
# for tri in triples[:100]:
#     try:
#         d=json.loads(tri[1])['triples']
#         for t in d:
#             if ('subject' not in t) or ('object' not in t) or ('relation' not in t):
#                 continue
#             dfg1['node_1'].append(t['subject'])
#             dfg1['node_2'].append(t['object'])
#             dfg1['edge'].append(t['relation'])
#             dfg1['chunk_id'].append(str(random.randint(0,1000000)))
#             dfg1['count'].append(4)
#     except KeyboardInterrupt:
#         break
#     except:
#         continue
    
# dfg1=pd.DataFrame(dfg1)
# dfg1.head()

,node_1,node_2,edge,count,chunk_id
0,cliente,conta,tem problema,4,366324
1,conta,em análise,esta,4,540512
2,atendente,solicitação interna,irá abrir,4,824164
3,atendente,e-mail,irá dar retorno,4,703686
4,cliente,fatura,tem problema,4,853622


In [9]:
dfg1.shape

(68, 5)

In [10]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
3645,verificação,pagamento,"44386,44386",2,contextual proximity
3648,verificação,pix,"44386,44386",2,contextual proximity
3652,verificação,senha,"44386,44386,44386",3,contextual proximity
3656,verificação,transação,"44386,44386",2,contextual proximity
3658,verificação,vencimento,"44386,44386",2,contextual proximity


In [11]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,11/12,análise,"44386,44386,44386",contextual proximity,3
1,11/12,aplicativo,"44386,44386",contextual proximity,2
2,11/12,aplicativo do banco,"44386,44386",contextual proximity,2
3,11/12,atendente,"44386,44386,44386,44386,44386,44386,44386,4438...",contextual proximity,14
4,11/12,atendimento ao cliente,"44386,44386,44386,44386,44386,44386,44386,4438...",contextual proximity,12
...,...,...,...,...,...
2256,verificação,pagamento,"44386,44386",contextual proximity,2
2257,verificação,pix,"44386,44386",contextual proximity,2
2258,verificação,senha,"44386,44386,44386",contextual proximity,3
2259,verificação,transação,"44386,44386",contextual proximity,2


Calculate the NetworkX Graph


In [12]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(61,)

In [13]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [14]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
# print(communities)

Number of Communities =  3


In [15]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,11/12,#5f57db,1
1,13/02,#57db5f,2
2,14/02,#db5f57,3
3,14/11,#db5f57,3
4,agendamento do pagamento,#db5f57,3
...,...,...,...
56,transação,#db5f57,3
57,transferência,#db5f57,3
58,vencimento,#db5f57,3
59,verificar,#db5f57,3


In [16]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [17]:
from pyvis.network import Network

graph_output_directory = "/home/kiki/Projects/BTG/graph/data_output/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

/home/kiki/Projects/BTG/graph/data_output/index.html


[GFX1-]: More than 2 GPUs detected via PCI, secondary GPU is arbitrary

